In [1]:
import pymysql
from collections import defaultdict
import pandas as pd
import numpy as np
import math

In [2]:
#Database connection
conn = pymysql.connect(user='root', password='root', database='acb', host='localhost')

#Load the data into a Pandas DataFrame
list_referees_cup = ['Jordi Aliaga', 'Martín Caballero', 'Calatrava', 'Antonio Conde', 'J.C García González', 'Daniel Hierrezuelo', 'Jiménez Trujillo', 'J.A. Martín Bertrán', 'M.A. Pérez Pérez', 'E. Pérez Pizarro', 'Carlos Peruga', 'Rafael Serrano']
dataframe_list = []
for referee in list_referees_cup:
    query = "SELECT game.referee_1,game.referee_2,game.referee_3, participant.* FROM participant RIGHT JOIN game ON game.id = participant.game_id WHERE game.referee_1 = '{0}' OR game.referee_2 = '{1}' OR game.referee_3 = '{2}'".format(referee, referee, referee)
    df = pd.read_sql(query, conn)
    fouls = df["fault"].sum()
    df_teams = df[df["display_name"] == 'Equipo']
    possessions = df_teams["possessions"].sum()
    fouls_possessions = fouls*100/possessions
    dataframe_list.append([referee, fouls_possessions])
    print(df.shape)
    print(referee)
    print(fouls_possessions)
    print("----------------------")

df_csv = pd.DataFrame(dataframe_list, columns=['Árbitro', 'Faltas por posesión'])
df_csv = df_csv.sort_values('Faltas por posesión', ascending = False)
print(df_csv.head(20))

(4003, 33)
Jordi Aliaga
57.318528035239055
----------------------
(1911, 33)
Martín Caballero
29.711528796864005
----------------------
(2378, 33)
Calatrava
30.77870872350912
----------------------
(11857, 33)
Antonio Conde
141.11809711385501
----------------------
(11578, 33)
J.C García González
140.87579865980987
----------------------
(12025, 33)
Daniel Hierrezuelo
139.0607611840641
----------------------
(11316, 33)
Jiménez Trujillo
129.9474222287133
----------------------
(11478, 33)
J.A. Martín Bertrán
140.5214557305812
----------------------
(11266, 33)
M.A. Pérez Pérez
138.14863506302683
----------------------
(11251, 33)
E. Pérez Pizarro
130.92020834906018
----------------------
(10382, 33)
Carlos Peruga
115.95744680851064
----------------------
(3897, 33)
Rafael Serrano
56.809804644305196
----------------------
                Árbitro  Faltas por posesión
3         Antonio Conde           141.118097
4   J.C García González           140.875799
7   J.A. Martín Bertrán         

In [3]:
# Faltas recibidas por posesión
query = "SELECT game.season, participant.* FROM participant LEFT JOIN game ON game.id = participant.game_id WHERE game.season = 2018"
df = pd.read_sql(query, conn)
print(df.shape)
df['games_played'] = np.where(df['minutes']>0, 1, 0)
df_perplayer = df.groupby(by=['display_name'], as_index=False).sum()
df_perplayer = df_perplayer[(df_perplayer["games_played"] > 10) & (df_perplayer["minutes"] > 12000)]

df_teams = df.groupby(by=['team_id'], as_index=False).sum()
df_teams


(5759, 31)


,team_id,season,id,game_id,actor_id,number,is_coach,is_starter,minutes,point,...,counterattack,block,received_block,dunk,fault,received_fault,plus_minus,efficiency,possessions,games_played
0,1,649796,6477107,255514,8045.0,4834.0,23,115,277500,2038,...,68,84,50,61,422,505,1080,2422,7173.0,266
1,2,649796,6550859,258202,22136.0,3977.0,23,115,277500,2005,...,69,48,35,68,463,480,1320,2354,6954.0,245
2,3,627598,6127210,241362,16754.0,4398.0,23,115,277500,1979,...,46,31,51,59,416,501,189,2248,6962.0,239
3,4,637688,6355116,250356,31146.0,6275.0,23,115,276000,1894,...,82,48,64,70,509,505,-85,1940,6666.0,241
4,5,639706,6553331,258276,57786.0,6463.0,23,115,277500,1795,...,19,54,54,39,467,472,-305,1951,6731.0,251
5,6,619526,5897030,232277,54540.0,4120.0,23,115,275999,1965,...,100,69,45,136,449,443,1510,2314,7130.0,234
6,7,649796,6567393,258790,61848.0,4659.0,23,115,277500,1898,...,52,54,54,45,480,457,-355,1984,6993.0,257
7,8,641724,6577631,259161,48720.0,4066.0,23,115,279000,1825,...,68,63,52,51,515,492,-1005,1791,6859.0,258
8,9,676030,9428589,372780,24021.0,3914.0,24,120,295500,1937,...,58,44,55,70,506,467,355,2066,6926.0,269
9,10,633652,6153060,242298,29801.0,5036.0,23,115,277500,1856,...,66,69,75,59,491,452,135,1973,7013.0,248


In [4]:
df_perplayer["steals_per_100_possessions"] = df_perplayer["steal"]*100/df_perplayer["possessions"]
df_perplayer = df_perplayer.sort_values('steals_per_100_possessions', ascending = False)
df_perplayer.head(10)

,display_name,season,id,game_id,team_id,actor_id,number,is_coach,is_starter,minutes,...,block,received_block,dunk,fault,received_fault,plus_minus,efficiency,possessions,games_played,steals_per_100_possessions
245,R. Luz,46414,469236,18495,92,2093.0,115.0,0,1,24769,...,2,3,1,57,32,37,137,424.0,23,6.839623
15,A. Hanga,34306,246723,9689,34,952.0,136.0,0,8,21720,...,3,1,9,36,43,61,201,416.0,17,6.730769
223,O. Jaramaz,40360,53389,1945,220,3000.0,640.0,0,8,14648,...,0,5,3,29,39,-55,83,245.0,18,6.122449
64,D. Hilliard,46414,467575,18426,138,4232.0,736.0,0,8,26983,...,1,4,6,45,48,188,233,526.0,23,5.703422
78,D. White,24216,634893,25219,108,3084.0,408.0,0,6,13024,...,1,1,0,28,28,41,85,248.0,11,5.645161
231,P. Garino,26234,245713,9648,78,3562.0,377.0,0,4,13376,...,0,1,2,23,11,90,75,289.0,12,5.536332
204,N. Brussino,46414,466430,18390,207,437.0,203.0,0,3,21006,...,3,2,6,53,18,-3,76,363.0,23,5.509642
173,L. Vildoza,46414,467349,18426,138,4002.0,69.0,0,14,29156,...,4,3,0,42,36,142,207,601.0,22,5.490849
314,X. Rabaseda,46414,466551,18392,391,1679.0,506.0,0,16,27296,...,8,7,8,40,42,76,199,521.0,23,5.374280
244,R. Fernández,36324,450763,17820,18,54.0,90.0,0,0,20118,...,0,0,1,24,30,137,154,398.0,18,5.025126


In [5]:
df_perplayer["steals_per_game"] = df_perplayer["steal"]/df_perplayer["games_played"]
df_perplayer = df_perplayer.sort_values('steals_per_game', ascending = False)
df_perplayer.head(10)

,display_name,season,id,game_id,team_id,actor_id,number,is_coach,is_starter,minutes,...,received_block,dunk,fault,received_fault,plus_minus,efficiency,possessions,games_played,steals_per_100_possessions,steals_per_game
15,A. Hanga,34306,246723,9689,34,952.0,136.0,0,8,21720,...,1,9,36,43,61,201,416.0,17,6.730769,1.647059
173,L. Vildoza,46414,467349,18426,138,4002.0,69.0,0,14,29156,...,3,0,42,36,142,207,601.0,22,5.490849,1.500000
221,O. Cook,46414,469080,18485,161,4715.0,230.0,0,23,42081,...,2,0,61,44,-8,251,834.0,23,3.717026,1.347826
231,P. Garino,26234,245713,9648,78,3562.0,377.0,0,4,13376,...,1,2,23,11,90,75,289.0,12,5.536332,1.333333
25,A. Renfroe,26234,42731,1558,156,2808.0,156.0,0,13,21205,...,1,1,31,44,-22,240,399.0,13,4.260652,1.307692
64,D. Hilliard,46414,467575,18426,138,4232.0,736.0,0,8,26983,...,4,6,45,48,188,233,526.0,23,5.703422,1.304348
78,D. White,24216,634893,25219,108,3084.0,408.0,0,6,13024,...,1,0,28,28,41,85,248.0,11,5.645161,1.272727
245,R. Luz,46414,469236,18495,92,2093.0,115.0,0,1,24769,...,3,1,57,32,37,137,424.0,23,6.839623,1.260870
1,A. Albicy,44396,467394,18426,88,2046.0,352.0,0,22,32227,...,7,0,44,58,-69,187,597.0,22,4.522613,1.227273
39,B. McCalebb,44396,465991,18374,286,4994.0,88.0,0,19,29906,...,14,0,32,42,-14,236,598.0,22,4.515050,1.227273


In [6]:
df_perplayer["fouls_drawn_per_100_possessions"] = df_perplayer["counterattack"]*100/df_perplayer["possessions"]
df_perplayer = df_perplayer.sort_values('fouls_drawn_per_100_possessions', ascending = False)
df_perplayer.head(10)

,display_name,season,id,game_id,team_id,actor_id,number,is_coach,is_starter,minutes,...,dunk,fault,received_fault,plus_minus,efficiency,possessions,games_played,steals_per_100_possessions,steals_per_game,fouls_drawn_per_100_possessions
180,M. Eriksson,36324,456354,18022,306,1224.0,144.0,0,1,21274,...,3,18,38,-38,193,429.0,18,2.331002,0.555556,4.895105
287,T. Shengelia,26234,40949,1492,78,2366.0,299.0,0,10,18284,...,18,19,68,123,253,333.0,13,3.903904,1.000000,4.804805
62,D. Ennis,46414,469438,18495,92,2300.0,713.0,0,22,36111,...,6,48,82,-11,297,669.0,23,2.840060,0.826087,3.886398
261,S. Llull,34306,442640,17521,17,153.0,391.0,0,13,23184,...,0,24,39,110,202,468.0,17,2.564103,0.705882,3.632479
93,F. Cruz,46414,470333,18531,184,3082.0,207.0,0,10,32290,...,0,57,44,-104,238,671.0,23,2.533532,0.739130,3.278689
50,C. Hannah,30270,451939,17861,255,3585.0,75.0,0,13,19882,...,0,38,49,22,166,400.0,15,3.000000,0.800000,3.250000
74,D. Strawberry,46414,466586,18392,391,1725.0,552.0,0,22,31828,...,3,67,58,-38,175,604.0,23,3.642384,0.956522,3.145695
102,G. Deck,42378,460357,18165,21,210.0,504.0,0,7,19627,...,3,21,27,63,131,403.0,21,1.488834,0.285714,2.729529
259,S. Dawson,36324,49201,1798,180,1422.0,18.0,0,13,25065,...,9,61,44,-1,183,487.0,18,2.464066,0.666667,2.669405
150,J. Voigtmann,44396,465993,18375,132,3872.0,154.0,0,12,30914,...,27,39,39,172,298,642.0,22,3.271028,0.954545,2.647975


In [7]:
df_perplayer["fouls_drawn_per_game"] = df_perplayer["counterattack"]/df_perplayer["games_played"]
df_perplayer = df_perplayer.sort_values('fouls_drawn_per_100_possessions', ascending = False)
df_perplayer.head(10)

,display_name,season,id,game_id,team_id,actor_id,number,is_coach,is_starter,minutes,...,fault,received_fault,plus_minus,efficiency,possessions,games_played,steals_per_100_possessions,steals_per_game,fouls_drawn_per_100_possessions,fouls_drawn_per_game
180,M. Eriksson,36324,456354,18022,306,1224.0,144.0,0,1,21274,...,18,38,-38,193,429.0,18,2.331002,0.555556,4.895105,1.166667
287,T. Shengelia,26234,40949,1492,78,2366.0,299.0,0,10,18284,...,19,68,123,253,333.0,13,3.903904,1.000000,4.804805,1.230769
62,D. Ennis,46414,469438,18495,92,2300.0,713.0,0,22,36111,...,48,82,-11,297,669.0,23,2.840060,0.826087,3.886398,1.130435
261,S. Llull,34306,442640,17521,17,153.0,391.0,0,13,23184,...,24,39,110,202,468.0,17,2.564103,0.705882,3.632479,1.000000
93,F. Cruz,46414,470333,18531,184,3082.0,207.0,0,10,32290,...,57,44,-104,238,671.0,23,2.533532,0.739130,3.278689,0.956522
50,C. Hannah,30270,451939,17861,255,3585.0,75.0,0,13,19882,...,38,49,22,166,400.0,15,3.000000,0.800000,3.250000,0.866667
74,D. Strawberry,46414,466586,18392,391,1725.0,552.0,0,22,31828,...,67,58,-38,175,604.0,23,3.642384,0.956522,3.145695,0.826087
102,G. Deck,42378,460357,18165,21,210.0,504.0,0,7,19627,...,21,27,63,131,403.0,21,1.488834,0.285714,2.729529,0.523810
259,S. Dawson,36324,49201,1798,180,1422.0,18.0,0,13,25065,...,61,44,-1,183,487.0,18,2.464066,0.666667,2.669405,0.722222
150,J. Voigtmann,44396,465993,18375,132,3872.0,154.0,0,12,30914,...,39,39,172,298,642.0,22,3.271028,0.954545,2.647975,0.772727


In [8]:
#  Assist Ratio is the number of assists a player averages per 100 possessions used
df_perplayer["assists_ratio"] = df_perplayer["assist"]*100/df_perplayer["possessions"]
df_perplayer = df_perplayer.sort_values('assists_ratio', ascending = False)
df_perplayer.head(8)

,display_name,season,id,game_id,team_id,actor_id,number,is_coach,is_starter,minutes,...,received_fault,plus_minus,efficiency,possessions,games_played,steals_per_100_possessions,steals_per_game,fouls_drawn_per_100_possessions,fouls_drawn_per_game,assists_ratio
282,T. Heurtel,46414,467915,18443,46,1357.0,299.0,0,8,26968,...,42,115,260,516.0,21,3.682171,0.904762,1.356589,0.333333,22.674419
236,P. Pozas,40360,458550,18098,100,4860.0,880.0,0,9,21176,...,42,-54,124,382.0,20,3.403141,0.650000,0.523560,0.100000,20.418848
70,D. Pérez,48432,676775,26749,384,2568.0,120.0,0,21,33756,...,79,-137,235,605.0,24,4.462810,1.125000,0.826446,0.208333,20.000000
25,A. Renfroe,26234,42731,1558,156,2808.0,156.0,0,13,21205,...,44,-22,240,399.0,13,4.260652,1.307692,2.255639,0.692308,19.548872
90,F. Bassas,48432,673547,26631,216,480.0,240.0,0,4,26093,...,43,30,185,457.0,24,2.625821,0.500000,1.531729,0.291667,19.474836
210,N. Laprovittola,46414,470182,18526,230,1886.0,230.0,0,23,41797,...,125,56,386,805.0,23,2.608696,0.913043,0.869565,0.304348,19.130435
184,M. Huertas,46414,467418,18426,138,4071.0,207.0,0,9,27825,...,32,128,236,576.0,23,3.125000,0.782609,1.562500,0.391304,19.097222
28,A. Sàbat,46414,468597,18472,115,4278.0,69.0,0,14,30651,...,42,14,183,573.0,23,2.443281,0.608696,0.698080,0.173913,18.673647


In [9]:
#  Assist Ratio is the number of assists a player averages per 100 possessions used
df_perplayer["assists_per_game"] = df_perplayer["assist"]/df_perplayer["games_played"]
df_perplayer = df_perplayer.sort_values('assists_ratio', ascending = False)
df_perplayer.head(8)

,display_name,season,id,game_id,team_id,actor_id,number,is_coach,is_starter,minutes,...,plus_minus,efficiency,possessions,games_played,steals_per_100_possessions,steals_per_game,fouls_drawn_per_100_possessions,fouls_drawn_per_game,assists_ratio,assists_per_game
282,T. Heurtel,46414,467915,18443,46,1357.0,299.0,0,8,26968,...,115,260,516.0,21,3.682171,0.904762,1.356589,0.333333,22.674419,5.571429
236,P. Pozas,40360,458550,18098,100,4860.0,880.0,0,9,21176,...,-54,124,382.0,20,3.403141,0.650000,0.523560,0.100000,20.418848,3.900000
70,D. Pérez,48432,676775,26749,384,2568.0,120.0,0,21,33756,...,-137,235,605.0,24,4.462810,1.125000,0.826446,0.208333,20.000000,5.041667
25,A. Renfroe,26234,42731,1558,156,2808.0,156.0,0,13,21205,...,-22,240,399.0,13,4.260652,1.307692,2.255639,0.692308,19.548872,6.000000
90,F. Bassas,48432,673547,26631,216,480.0,240.0,0,4,26093,...,30,185,457.0,24,2.625821,0.500000,1.531729,0.291667,19.474836,3.708333
210,N. Laprovittola,46414,470182,18526,230,1886.0,230.0,0,23,41797,...,56,386,805.0,23,2.608696,0.913043,0.869565,0.304348,19.130435,6.695652
184,M. Huertas,46414,467418,18426,138,4071.0,207.0,0,9,27825,...,128,236,576.0,23,3.125000,0.782609,1.562500,0.391304,19.097222,4.782609
28,A. Sàbat,46414,468597,18472,115,4278.0,69.0,0,14,30651,...,14,183,573.0,23,2.443281,0.608696,0.698080,0.173913,18.673647,4.652174


In [10]:
#PACE:  The number of possessions per 40 minutes for a team or player.
df_perplayer["pace"] = df_perplayer["possessions"]*2400/df_perplayer["minutes"]
df_perplayer = df_perplayer.sort_values('pace', ascending = False)
df_perplayer

,display_name,season,id,game_id,team_id,actor_id,number,is_coach,is_starter,minutes,...,efficiency,possessions,games_played,steals_per_100_possessions,steals_per_game,fouls_drawn_per_100_possessions,fouls_drawn_per_game,assists_ratio,assists_per_game,pace
231,P. Garino,26234,245713,9648,78,3562.0,377.0,0,4,13376,...,75,289.0,12,5.536332,1.333333,2.076125,0.500000,2.768166,0.666667,51.854067
93,F. Cruz,46414,470333,18531,184,3082.0,207.0,0,10,32290,...,238,671.0,23,2.533532,0.739130,3.278689,0.956522,8.345753,2.434783,49.873026
150,J. Voigtmann,44396,465993,18375,132,3872.0,154.0,0,12,30914,...,298,642.0,22,3.271028,0.954545,2.647975,0.772727,7.632399,2.227273,49.841496
275,S. Yusta,40360,459802,18147,20,3380.0,320.0,0,10,16214,...,124,336.0,16,2.678571,0.562500,0.892857,0.187500,4.166667,0.875000,49.734797
184,M. Huertas,46414,467418,18426,138,4071.0,207.0,0,9,27825,...,236,576.0,23,3.125000,0.782609,1.562500,0.391304,19.097222,4.782609,49.681941
154,K. Prepelic,40360,251666,9866,20,220.0,500.0,0,2,12588,...,55,260.0,19,0.384615,0.052632,0.384615,0.052632,6.538462,0.894737,49.571020
173,L. Vildoza,46414,467349,18426,138,4002.0,69.0,0,14,29156,...,207,601.0,22,5.490849,1.500000,1.164725,0.318182,10.316140,2.818182,49.471807
125,J. Fernández,38342,51072,1867,57,779.0,57.0,0,9,28176,...,313,579.0,19,3.626943,1.105263,1.554404,0.473684,16.753022,5.105263,49.318569
102,G. Deck,42378,460357,18165,21,210.0,504.0,0,7,19627,...,131,403.0,21,1.488834,0.285714,2.729529,0.523810,5.955335,1.142857,49.279054
68,D. Navarro,46414,468750,18472,115,4439.0,460.0,0,1,15691,...,127,322.0,22,1.863354,0.272727,0.621118,0.090909,13.354037,1.954545,49.251163


In [11]:
#FG%:
df_perplayer["FG%"] = (df_perplayer["t2"]+df_perplayer["t3"])/ (df_perplayer["t2_attempt"] + df_perplayer["t3_attempt"])
df_perplayer = df_perplayer.sort_values('FG%', ascending = False)
df_perplayer.head(10)

,display_name,season,id,game_id,team_id,actor_id,number,is_coach,is_starter,minutes,...,possessions,games_played,steals_per_100_possessions,steals_per_game,fouls_drawn_per_100_possessions,fouls_drawn_per_game,assists_ratio,assists_per_game,pace,FG%
309,W. Tavares,44396,459042,18119,22,176.0,484.0,0,18,25689,...,498.0,21,1.606426,0.380952,0.200803,0.047619,2.610442,0.619048,46.525750,0.761364
220,O. Balvin,46414,466464,18392,391,1587.0,276.0,0,9,26028,...,484.0,23,2.892562,0.608696,0.000000,0.000000,4.545455,0.956522,44.628861,0.662500
51,C. Iverson,48432,673464,26631,216,384.0,96.0,0,21,34638,...,607.0,24,3.130148,0.791667,0.988468,0.250000,3.130148,0.791667,42.057856,0.652941
81,E. Cate,46414,467666,18433,322,2829.0,345.0,0,1,19223,...,332.0,23,1.807229,0.260870,0.602410,0.086957,1.807229,0.260870,41.450346,0.651515
108,G. Shermadini,46414,469117,18486,69,1058.0,391.0,0,8,26913,...,498.0,23,2.409639,0.521739,0.602410,0.130435,2.811245,0.608696,44.409765,0.642857
29,A. Tomic,42378,462024,18224,42,1344.0,924.0,0,17,25566,...,476.0,21,2.521008,0.571429,0.000000,0.000000,7.563025,1.714286,44.684346,0.633803
155,K. Seraphin,42378,462656,18256,42,5313.0,21.0,0,3,17235,...,353.0,19,1.699717,0.315789,1.416431,0.263158,5.382436,1.000000,49.155788,0.626263
308,V. Poirier,46414,467498,18426,138,4163.0,391.0,0,20,28392,...,577.0,23,3.292894,0.826087,1.213172,0.304348,4.159445,1.043478,48.774303,0.625000
159,K. Tumba,44396,260772,10198,308,2750.0,528.0,0,16,18962,...,314.0,22,3.184713,0.454545,0.318471,0.045455,1.910828,0.272727,39.742643,0.622222
200,M. Todorovic,46414,470205,18526,230,1909.0,253.0,0,13,31796,...,591.0,23,3.045685,0.782609,0.846024,0.217391,9.475465,2.434783,44.609385,0.621212


In [12]:
#eFG%: ((FGM + (0.5 * 3PM)) / FGA
df_perplayer["eFG%"] = (df_perplayer["t2"]+df_perplayer["t3"]+(0.5*df_perplayer["t3"])) / (df_perplayer["t2_attempt"] + df_perplayer["t3_attempt"])
df_perplayer = df_perplayer.sort_values('eFG%', ascending = False)
df_perplayer.head(10)

,display_name,season,id,game_id,team_id,actor_id,number,is_coach,is_starter,minutes,...,games_played,steals_per_100_possessions,steals_per_game,fouls_drawn_per_100_possessions,fouls_drawn_per_game,assists_ratio,assists_per_game,pace,FG%,eFG%
309,W. Tavares,44396,459042,18119,22,176.0,484.0,0,18,25689,...,21,1.606426,0.380952,0.200803,0.047619,2.610442,0.619048,46.525750,0.761364,0.761364
231,P. Garino,26234,245713,9648,78,3562.0,377.0,0,4,13376,...,12,5.536332,1.333333,2.076125,0.500000,2.768166,0.666667,51.854067,0.576923,0.711538
148,J. Taylor,36324,450535,17806,18,216.0,792.0,0,7,16288,...,17,2.380952,0.411765,2.380952,0.411765,4.421769,0.764706,43.320236,0.583333,0.673611
167,L. Labeyrie,42378,461977,18229,378,609.0,147.0,0,4,20541,...,21,2.337662,0.428571,1.558442,0.285714,3.636364,0.666667,44.983204,0.611765,0.670588
220,O. Balvin,46414,466464,18392,391,1587.0,276.0,0,9,26028,...,23,2.892562,0.608696,0.000000,0.000000,4.545455,0.956522,44.628861,0.662500,0.662500
51,C. Iverson,48432,673464,26631,216,384.0,96.0,0,21,34638,...,24,3.130148,0.791667,0.988468,0.250000,3.130148,0.791667,42.057856,0.652941,0.655882
161,K. Wiltjer,46414,469209,18486,69,1150.0,759.0,0,22,26523,...,23,0.785855,0.173913,0.589391,0.130435,3.536346,0.782609,46.058138,0.525253,0.651515
81,E. Cate,46414,467666,18433,322,2829.0,345.0,0,1,19223,...,23,1.807229,0.260870,0.602410,0.086957,1.807229,0.260870,41.450346,0.651515,0.651515
108,G. Shermadini,46414,469117,18486,69,1058.0,391.0,0,8,26913,...,23,2.409639,0.521739,0.602410,0.130435,2.811245,0.608696,44.409765,0.642857,0.642857
44,C. Alocén,46414,468240,18451,299,5382.0,276.0,0,1,17244,...,21,4.913295,0.809524,0.578035,0.095238,11.271676,1.857143,48.155880,0.527778,0.638889


In [13]:
df_perplayer["lastre_FG%"] = df_perplayer["eFG%"] - df_perplayer["FG%"]
df_perplayer = df_perplayer.sort_values('lastre_FG%', ascending = False)
df_perplayer.head(10)

,display_name,season,id,game_id,team_id,actor_id,number,is_coach,is_starter,minutes,...,steals_per_100_possessions,steals_per_game,fouls_drawn_per_100_possessions,fouls_drawn_per_game,assists_ratio,assists_per_game,pace,FG%,eFG%,lastre_FG%
7,A. Corbacho,46414,469919,18510,368,2645.0,759.0,0,6,24646,...,2.017937,0.409091,0.672646,0.136364,1.793722,0.363636,43.430983,0.394958,0.584034,0.189076
72,D. Rudez,46414,467612,18433,322,2783.0,230.0,0,2,17671,...,2.312139,0.347826,0.578035,0.086957,4.624277,0.695652,46.992247,0.415842,0.589109,0.173267
160,K. Vasileiadis,42378,261140,10209,105,4137.0,1617.0,0,17,31104,...,2.047782,0.571429,0.170648,0.047619,7.679181,2.142857,45.216049,0.394231,0.562500,0.168269
10,A. Doornekamp,46414,467141,18410,414,828.0,966.0,0,13,26348,...,2.991453,0.608696,0.641026,0.130435,6.196581,1.260870,42.629422,0.435294,0.600000,0.164706
172,L. Staiger,46414,671978,26577,207,345.0,23.0,0,6,21414,...,1.612903,0.272727,1.075269,0.181818,9.408602,1.590909,41.692351,0.364486,0.523364,0.158879
19,A. Obst,42378,464962,18335,105,3990.0,273.0,0,7,19432,...,2.500000,0.450000,0.000000,0.000000,5.833333,1.050000,44.462742,0.462963,0.620370,0.157407
43,B. Simons,36324,449983,17792,90,3366.0,126.0,0,6,21306,...,2.025316,0.444444,0.253165,0.055556,4.556962,1.000000,44.494509,0.416058,0.572993,0.156934
28,A. Sàbat,46414,468597,18472,115,4278.0,69.0,0,14,30651,...,2.443281,0.608696,0.698080,0.173913,18.673647,4.652174,44.866399,0.373737,0.530303,0.156566
285,T. McFadden,32288,253459,9931,145,352.0,205.0,0,11,20783,...,2.680965,0.625000,1.608579,0.375000,10.455764,2.437500,43.073666,0.485294,0.636029,0.150735
268,S. Salin,46414,469046,18486,69,989.0,230.0,0,7,31516,...,1.677852,0.434783,0.335570,0.086957,4.697987,1.217391,45.386470,0.366013,0.516340,0.150327


In [14]:
#TS%: Points/ [2*(Field Goals Attempted+0.44*Free Throws Attempted)]
df_perplayer["TS%"] = (df_perplayer["point"])/((2*(df_perplayer["t2_attempt"] + df_perplayer["t3_attempt"]))+(0.44*df_perplayer["t1_attempt"]))
df_perplayer = df_perplayer.sort_values('TS%', ascending = False)
df_perplayer.head()

,display_name,season,id,game_id,team_id,actor_id,number,is_coach,is_starter,minutes,...,steals_per_game,fouls_drawn_per_100_possessions,fouls_drawn_per_game,assists_ratio,assists_per_game,pace,FG%,eFG%,lastre_FG%,TS%
309,W. Tavares,44396,459042,18119,22,176.0,484.0,0,18,25689,...,0.380952,0.200803,0.047619,2.610442,0.619048,46.525750,0.761364,0.761364,0.0,0.847722
178,M. Diagne,46414,469313,18495,92,2162.0,483.0,0,18,24732,...,0.681818,0.913242,0.181818,2.283105,0.454545,42.503639,0.562500,0.562500,0.0,0.780294
108,G. Shermadini,46414,469117,18486,69,1058.0,391.0,0,8,26913,...,0.521739,0.602410,0.130435,2.811245,0.608696,44.409765,0.642857,0.642857,0.0,0.766773
81,E. Cate,46414,467666,18433,322,2829.0,345.0,0,1,19223,...,0.260870,0.602410,0.086957,1.807229,0.260870,41.450346,0.651515,0.651515,0.0,0.755328
220,O. Balvin,46414,466464,18392,391,1587.0,276.0,0,9,26028,...,0.608696,0.000000,0.000000,4.545455,0.956522,44.628861,0.662500,0.662500,0.0,0.747177


In [15]:
#Turnover ratio: The number of turnovers a player or team averages per 100 possessions used
df_perplayer["turnover_per_game"] = df_perplayer["turnover"]/df_perplayer["games_played"]
df_perplayer = df_perplayer.sort_values('turnover_per_game', ascending = False)
df_perplayer.head()

,display_name,season,id,game_id,team_id,actor_id,number,is_coach,is_starter,minutes,...,fouls_drawn_per_100_possessions,fouls_drawn_per_game,assists_ratio,assists_per_game,pace,FG%,eFG%,lastre_FG%,TS%,turnover_per_game
210,N. Laprovittola,46414,470182,18526,230,1886.0,230.0,0,23,41797,...,0.869565,0.304348,19.130435,6.695652,46.223413,0.448148,0.548148,0.100000,0.644347,4.652174
25,A. Renfroe,26234,42731,1558,156,2808.0,156.0,0,13,21205,...,2.255639,0.692308,19.548872,6.000000,45.159161,0.520325,0.638211,0.117886,0.702441,2.692308
36,B. Dubljevic,40360,463862,18294,360,640.0,280.0,0,15,28070,...,0.927644,0.250000,6.493506,1.750000,46.084788,0.515625,0.585938,0.070312,0.649892,2.450000
13,A. Gentile,36324,451647,17852,126,3618.0,90.0,0,16,27191,...,1.270417,0.388889,5.444646,1.666667,48.633739,0.481013,0.495781,0.014768,0.545965,2.333333
259,S. Dawson,36324,49201,1798,180,1422.0,18.0,0,13,25065,...,2.669405,0.722222,5.544148,1.500000,46.630760,0.490196,0.568627,0.078431,0.645439,2.277778


In [16]:
#Turnover ratio: The number of turnovers a player or team averages per 100 possessions used
df_perplayer["turnover_ratio"] = df_perplayer["turnover"]*100/df_perplayer["possessions"]
df_perplayer = df_perplayer.sort_values('turnover_ratio', ascending = False)
df_perplayer.head()

,display_name,season,id,game_id,team_id,actor_id,number,is_coach,is_starter,minutes,...,fouls_drawn_per_game,assists_ratio,assists_per_game,pace,FG%,eFG%,lastre_FG%,TS%,turnover_per_game,turnover_ratio
210,N. Laprovittola,46414,470182,18526,230,1886.0,230.0,0,23,41797,...,0.304348,19.130435,6.695652,46.223413,0.448148,0.548148,0.100000,0.644347,4.652174,13.291925
206,N. Dimitrijevic,46414,470101,18526,230,1794.0,99.0,0,0,20785,...,0.391304,12.878788,2.217391,45.725283,0.478261,0.526087,0.047826,0.630048,1.739130,10.101010
129,J. Gutiérrez,36324,48833,1785,288,1872.0,18.0,0,4,17455,...,0.352941,13.479624,2.529412,43.861358,0.360360,0.387387,0.027027,0.474415,1.882353,10.031348
36,B. Dubljevic,40360,463862,18294,360,640.0,280.0,0,15,28070,...,0.250000,6.493506,1.750000,46.084788,0.515625,0.585938,0.070312,0.649892,2.450000,9.090909
25,A. Renfroe,26234,42731,1558,156,2808.0,156.0,0,13,21205,...,0.692308,19.548872,6.000000,45.159161,0.520325,0.638211,0.117886,0.702441,2.692308,8.771930


In [17]:
#Turnover ratio: The number of turnovers a player or team averages per 100 possessions used
df_perplayer["assist_turnover"] = df_perplayer["assist"]/df_perplayer["turnover"]
df_perplayer = df_perplayer.sort_values('assist_turnover', ascending = False)
df_perplayer.head(7)

,display_name,season,id,game_id,team_id,actor_id,number,is_coach,is_starter,minutes,...,assists_ratio,assists_per_game,pace,FG%,eFG%,lastre_FG%,TS%,turnover_per_game,turnover_ratio,assist_turnover
221,O. Cook,46414,469080,18485,161,4715.0,230.0,0,23,42081,...,17.266187,6.260870,47.565410,0.371212,0.488636,0.117424,0.570858,1.521739,4.196643,4.114286
236,P. Pozas,40360,458550,18098,100,4860.0,880.0,0,9,21176,...,20.418848,3.900000,43.294295,0.263889,0.361111,0.097222,0.459369,1.100000,5.759162,3.545455
28,A. Sàbat,46414,468597,18472,115,4278.0,69.0,0,14,30651,...,18.673647,4.652174,44.866399,0.373737,0.530303,0.156566,0.597072,1.391304,5.584642,3.343750
91,F. Campazzo,42378,461183,18200,21,84.0,147.0,0,11,30568,...,17.910448,5.142857,47.343627,0.379310,0.451724,0.072414,0.605364,1.571429,5.472637,3.272727
237,P. Ribas,46414,467813,18443,46,1242.0,115.0,0,5,18945,...,13.054830,2.777778,48.519398,0.468085,0.617021,0.148936,0.675813,0.888889,4.177546,3.125000
278,T. Bellas,44396,468113,18451,176,2904.0,154.0,0,18,29189,...,9.633911,2.272727,42.673610,0.424000,0.512000,0.088000,0.593195,0.772727,3.275530,2.941176
68,D. Navarro,46414,468750,18472,115,4439.0,460.0,0,1,15691,...,13.354037,1.954545,49.251163,0.423729,0.474576,0.050847,0.598942,0.681818,4.658385,2.866667


In [23]:
df_perplayer[df_perplayer['display_name'] == 'T. Heurtel']

,display_name,season,id,game_id,team_id,actor_id,number,is_coach,is_starter,minutes,...,assists_per_game,pace,FG%,eFG%,lastre_FG%,TS%,turnover_per_game,turnover_ratio,assist_turnover,PER
282,T. Heurtel,46414,467915,18443,46,1357.0,299.0,0,8,26968,...,5.571429,45.921092,0.506173,0.555556,0.049383,0.617681,2.0,8.139535,2.785714,8.885714


In [19]:
#PER Hollinger: PTS + 0.4 * FG - 0.7 * FGA - 0.4*(FTA - FT) + 0.7 * ORB + 0.3 * DRB + STL + 0.7 * AST + 0.7 * BLK - 0.4 * PF - TOV.
df_perplayer["PER"] = (df_perplayer["point"]+(0.4*(df_perplayer["t2"]+df_perplayer["t3"]))-(0.7*(df_perplayer["t2_attempt"]+df_perplayer["t3_attempt"]))\
-(0.4*(df_perplayer["t1_attempt"]-df_perplayer["t1"]))+(0.7*df_perplayer["offensive_reb"])\
+(0.3*df_perplayer["defensive_reb"])+df_perplayer["steal"]+(0.7*df_perplayer["assist"])+(0.7*df_perplayer["block"])\
-(0.4*df_perplayer["fault"])-df_perplayer["turnover"])/df_perplayer["games_played"]
df_perplayer = df_perplayer.sort_values('PER', ascending = False)
df_perplayer.head(7)

,display_name,season,id,game_id,team_id,actor_id,number,is_coach,is_starter,minutes,...,assists_per_game,pace,FG%,eFG%,lastre_FG%,TS%,turnover_per_game,turnover_ratio,assist_turnover,PER
25,A. Renfroe,26234,42731,1558,156,2808.0,156.0,0,13,21205,...,6.000000,45.159161,0.520325,0.638211,0.117886,0.702441,2.692308,8.771930,2.228571,12.738462
287,T. Shengelia,26234,40949,1492,78,2366.0,299.0,0,10,18284,...,2.538462,43.710348,0.554688,0.601562,0.046875,0.684835,1.384615,5.405405,1.833333,12.461538
160,K. Vasileiadis,42378,261140,10209,105,4137.0,1617.0,0,17,31104,...,2.142857,45.216049,0.394231,0.562500,0.168269,0.699500,1.142857,4.095563,1.875000,10.866667
210,N. Laprovittola,46414,470182,18526,230,1886.0,230.0,0,23,41797,...,6.695652,46.223413,0.448148,0.548148,0.100000,0.644347,4.652174,13.291925,1.439252,10.817391
125,J. Fernández,38342,51072,1867,57,779.0,57.0,0,9,28176,...,5.105263,49.318569,0.461957,0.532609,0.070652,0.623981,1.789474,5.872193,2.852941,10.678947
118,J. Beirán,44396,259985,10168,198,550.0,726.0,0,21,37218,...,3.818182,44.688054,0.481132,0.577830,0.096698,0.612992,1.500000,4.761905,2.545455,10.677273
58,D. Brizuela,46414,469034,18485,161,4669.0,184.0,0,19,33635,...,2.652174,48.734949,0.496154,0.555769,0.059615,0.646120,1.956522,6.588580,1.355556,10.447826


In [20]:
df_perplayer[df_perplayer['display_name'] == 'F. Campazzo']['pace']

91    47.343627
Name: pace, dtype: float64

In [24]:
df_perplayer[df_perplayer['display_name'] == 'T. Heurtel']['pace']

282    45.921092
Name: pace, dtype: float64

In [ ]:
df_baskonia = df[df["team_id"] == 6]
(df_baskonia['counterattack'].sum())/(df_baskonia[df_baskonia['display_name'] == 'Equipo']['possessions'].sum())

In [ ]:
df_baskonia = df[df["team_id"] == 1]
(df_baskonia['counterattack'].sum())/(df_baskonia[df_baskonia['display_name'] == 'Equipo']['possessions'].sum())

In [ ]:
df_baskonia = df[df["team_id"] == 2]
(df_baskonia['counterattack'].sum())/(df_baskonia[df_baskonia['display_name'] == 'Equipo']['possessions'].sum())

In [ ]:
df_baskonia = df[df["team_id"] == 3]
(df_baskonia['counterattack'].sum())/(df_baskonia[df_baskonia['display_name'] == 'Equipo']['possessions'].sum())

In [ ]:
df_baskonia = df[df["team_id"] == 8]
(df_baskonia['counterattack'].sum())/(df_baskonia[df_baskonia['display_name'] == 'Equipo']['possessions'].sum())

In [ ]:
df_baskonia = df[df["team_id"] == 9]
(df_baskonia['counterattack'].sum())/(df_baskonia[df_baskonia['display_name'] == 'Equipo']['possessions'].sum())

In [ ]:
df_baskonia = df[df["team_id"] == 13]
(df_baskonia['counterattack'].sum())/(df_baskonia[df_baskonia['display_name'] == 'Equipo']['possessions'].sum())

In [ ]:
df_baskonia = df[df["team_id"] == 18]
(df_baskonia['counterattack'].sum())/(df_baskonia[df_baskonia['display_name'] == 'Equipo']['possessions'].sum())